# Time Stamp Extract

Brief 1-2 sentence description of notebook.

TODO: Supplement the description
- Notebook that extracts the timestamps and gets the time that tones played

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
import utilities.helper
import trodes.read_exported

# Functions

In [8]:
# Function to filter rows where start/stop range overlap with the specified range
def filter_overlapping_ranges(frame_ranges, start=0, stop=np.inf):
    # Check for overlap
    overlaps = np.logical_or(np.logical_and(frame_ranges[:,0] < stop, frame_ranges[:,0] >= start),
                             np.logical_and(frame_ranges[:,1] <= stop, frame_ranges[:,1] > start))
    return frame_ranges[overlaps]

## Inputs & Data

- Explanation of each input and where it comes from.

Inputs and Required data loading
- input variable names are in all caps snake case
- Whenever an input changes or is used for processing 
- The variables are all lower in snake case

In [9]:
# Path of the directory that contains the Spike Gadgets recording and the exported timestamp files
# Exported with this tool https://docs.spikegadgets.com/en/latest/basic/ExportFunctions.html
# Export these files:
    # -raw – Continuous raw band export.
    # -dio – Digital IO channel state change export.
    # -analogio – Continuous analog IO export.
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_novel_comp"

In [10]:
TRODES_METADATA_DF = pd.read_pickle("./proc/{}_00_trodes_metadata.pkl".format(OUTPUT_PREFIX))

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Other documentation

raw directory
- raw_group0.dat
    - voltage_value: Array with voltage measurement for each channel at each timestamp
- timestamps.dat
    - voltage_time_stamp: The time stamp of each voltage measurement

parent directory
- 1.videoTimeStamps.cameraHWSync
    - frame_number: Calculated by getting the index of each video time stamp tuple 
    - PosTimestamp: The time stamp of each video frame
    - HWframeCount: Unknown value. Starts at 30742 and increases by 1 for each tuple  
    - HWTimestamp: Unknown value. All zeroes
    - video_time: Calculated by dividing the frame number by the fps(frames per second) 
    - video_seconds: video_time, but rounded to seconds  	
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_PosTimestamp 	
        - filledHWframeCount 	
        - filled_frame_number 	
        - filled_video_time 	
        - filled_video_seconds 	

DIO directory
- dio_ECU_Din1.dat
    - time: The time stamp the corresponds to the DIN input
    - state: Binary state of whether there is input from DIN or not 	
    - trial_number: Calculated by adding 1 to every time there is a DIN input
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_state 	
        - filled_trial_number

ss_output directory (Spike sorting with Spike interface)
- firings.npz
    - unit_id: All the units that had a spike train for the given timestamp 	
    - number_of_units: Calculated by counting the number of units that had a spike train

## Functions

- function names are short and in snake case all lowercase
- a function name should be unique but does not have to describe the function
- doc strings describe functions not function names

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [11]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

In [12]:
TRODES_METADATA_DF = TRODES_METADATA_DF.drop(columns=["current_subject", "video_timestamps", "recording", "session_path"], errors="ignore")
TRODES_METADATA_DF = TRODES_METADATA_DF.drop_duplicates(subset=["video_name"])

In [13]:
exploded_TRODES_METADATA_DF = TRODES_METADATA_DF.explode(column = ["tone_frames", "tone_timestamps"])

In [14]:
exploded_TRODES_METADATA_DF

,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20240401_151442_comp_novel_subj_3-1_and_3-3_an...,"[0, 83]","[[0, 83], [84, 118], [158, 205], [275, 277], [...","[[0, 83], [364, 372], [474, 481], [625, 628], ...",20240401_151442_comp_novel_subj_3-1_and_3-3_an...,"[3.1, 3.3, 4.2, 4.4]",2932501,66530053,"[0, 84230]","[[0, 84230], [84627, 119430], [158430, 206033]...","[[0, 84230], [365233, 373033], [476236, 483234..."
0,20240401_151442_comp_novel_subj_3-1_and_3-3_an...,"[1280, 1481]","[[0, 83], [84, 118], [158, 205], [275, 277], [...","[[0, 83], [364, 372], [474, 481], [625, 628], ...",20240401_151442_comp_novel_subj_3-1_and_3-3_an...,"[3.1, 3.3, 4.2, 4.4]",2932501,66530053,"[1284645, 1484649]","[[0, 84230], [84627, 119430], [158430, 206033]...","[[0, 84230], [365233, 373033], [476236, 483234..."
0,20240401_151442_comp_novel_subj_3-1_and_3-3_an...,"[3278, 3477]","[[0, 83], [84, 118], [158, 205], [275, 277], [...","[[0, 83], [364, 372], [474, 481], [625, 628], ...",20240401_151442_comp_novel_subj_3-1_and_3-3_an...,"[3.1, 3.3, 4.2, 4.4]",2932501,66530053,"[3284672, 3484671]","[[0, 84230], [84627, 119430], [158430, 206033]...","[[0, 84230], [365233, 373033], [476236, 483234..."
0,20240401_151442_comp_novel_subj_3-1_and_3-3_an...,"[4476, 4675]","[[0, 83], [84, 118], [158, 205], [275, 277], [...","[[0, 83], [364, 372], [474, 481], [625, 628], ...",20240401_151442_comp_novel_subj_3-1_and_3-3_an...,"[3.1, 3.3, 4.2, 4.4]",2932501,66530053,"[4484687, 4684691]","[[0, 84230], [84627, 119430], [158430, 206033]...","[[0, 84230], [365233, 373033], [476236, 483234..."
0,20240401_151442_comp_novel_subj_3-1_and_3-3_an...,"[5474, 5673]","[[0, 83], [84, 118], [158, 205], [275, 277], [...","[[0, 83], [364, 372], [474, 481], [625, 628], ...",20240401_151442_comp_novel_subj_3-1_and_3-3_an...,"[3.1, 3.3, 4.2, 4.4]",2932501,66530053,"[5484697, 5684696]","[[0, 84230], [84627, 119430], [158430, 206033]...","[[0, 84230], [365233, 373033], [476236, 483234..."
...,...,...,...,...,...,...,...,...,...,...,...
69,20240412_161135_comp_novel_subj_4-2_and_4-4_an...,"[55593, 55794]","[[1, 114], [114, 115], [116, 145], [546, 609],...","[[1, 114], [118, 119], [621, 642], [657, 670],...",20240412_161135_comp_novel_subj_4-2_and_4-4_an...,"[4.2, 4.4, 5.2, 5.3]",4542881,67770406,"[55613392, 55813392]","[[0, 112297], [112694, 113891], [115894, 14549...","[[0, 112297], [116491, 117892], [620901, 64270..."
69,20240412_161135_comp_novel_subj_4-2_and_4-4_an...,"[56592, 56791]","[[1, 114], [114, 115], [116, 145], [546, 609],...","[[1, 114], [118, 119], [621, 642], [657, 670],...",20240412_161135_comp_novel_subj_4-2_and_4-4_an...,"[4.2, 4.4, 5.2, 5.3]",4542881,67770406,"[56613404, 56813407]","[[0, 112297], [112694, 113891], [115894, 14549...","[[0, 112297], [116491, 117892], [620901, 64270..."
69,20240412_161135_comp_novel_subj_4-2_and_4-4_an...,"[57989, 58188]","[[1, 114], [114, 115], [116, 145], [546, 609],...","[[1, 114], [118, 119], [621, 642], [657, 670],...",20240412_161135_comp_novel_subj_4-2_and_4-4_an...,"[4.2, 4.4, 5.2, 5.3]",4542881,67770406,"[58013422, 58213425]","[[0, 112297], [112694, 113891], [115894, 14549...","[[0, 112297], [116491, 117892], [620901, 64270..."
69,20240412_161135_comp_novel_subj_4-2_and_4-4_an...,"[60185, 60385]","[[1, 114], [114, 115], [116, 145], [546, 609],...","[[1, 114], [118, 119], [621, 642], [657, 670],...",20240412_161135_comp_novel_subj_4-2_and_4-4_an...,"[4.2, 4.4, 5.2, 5.3]",4542881,67770406,"[60213450, 60413452]","[[0, 112297], [112694, 113891], [115894, 14549...","[[0, 112297], [116491, 117892], [620901, 64270..."


In [15]:
for entry_col in [col for col in exploded_TRODES_METADATA_DF.columns if "port_entry" in col]:
    print(entry_col)
    exploded_TRODES_METADATA_DF[entry_col] = exploded_TRODES_METADATA_DF[entry_col].apply(lambda x: np.array(x))


box_1_port_entry_frames
box_2_port_entry_frames
box_1_port_entry_timestamps
box_2_port_entry_timestamps


In [16]:
columns_to_update = ["box_1_port_entry_frames", "box_2_port_entry_frames", "box_1_port_entry_timestamps", "box_2_port_entry_timestamps"]
tone_columns = ["tone_frames", "tone_frames", "tone_timestamps", "tone_timestamps"]

for col, tone_col in zip(columns_to_update, tone_columns):
    exploded_TRODES_METADATA_DF[col] = exploded_TRODES_METADATA_DF.apply(lambda x: filter_overlapping_ranges(x[col], x[tone_col][0], x[tone_col][1]), axis=1)

In [17]:
exploded_TRODES_METADATA_DF["tone_start_frame"] = exploded_TRODES_METADATA_DF["tone_frames"].apply(lambda x: x[0])
exploded_TRODES_METADATA_DF["tone_stop_frame"] = exploded_TRODES_METADATA_DF["tone_frames"].apply(lambda x: x[1])

In [18]:
exploded_TRODES_METADATA_DF["tone_start_timestamp"] = exploded_TRODES_METADATA_DF["tone_timestamps"].apply(lambda x: x[0])
exploded_TRODES_METADATA_DF["tone_stop_timestamp"] = exploded_TRODES_METADATA_DF["tone_timestamps"].apply(lambda x: x[1])


In [19]:
exploded_TRODES_METADATA_DF = exploded_TRODES_METADATA_DF.drop(columns=["tone_frames", "tone_timestamps"], errors="ignore")

- Create a new column for start and stop

In [20]:
exploded_TRODES_METADATA_DF = exploded_TRODES_METADATA_DF[sorted(exploded_TRODES_METADATA_DF.columns, key=lambda s: s.split('_')[-1])].copy()

In [21]:
exploded_TRODES_METADATA_DF.to_pickle("./proc/{}_per_video_trial_labels.pkl".format(OUTPUT_PREFIX))

- Exporting to CSV

In [22]:
for entry_col in [col for col in exploded_TRODES_METADATA_DF.columns if "port_entry" in col]:
    print(entry_col)
    exploded_TRODES_METADATA_DF[entry_col] = exploded_TRODES_METADATA_DF[entry_col].apply(lambda x: ' '.join(map(str, x)))

box_1_port_entry_frames
box_2_port_entry_frames
box_1_port_entry_timestamps
box_2_port_entry_timestamps


In [23]:
exploded_TRODES_METADATA_DF.head()

,session_dir,tone_start_frame,tone_stop_frame,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20240401_151442_comp_novel_subj_3-1_and_3-3_an...,0,83,[ 0 83],[ 0 83],20240401_151442_comp_novel_subj_3-1_and_3-3_an...,"[3.1, 3.3, 4.2, 4.4]",2932501,66530053,0,84230,[ 0 84230],[ 0 84230]
0,20240401_151442_comp_novel_subj_3-1_and_3-3_an...,1280,1481,[1302 1322],[1414 1419],20240401_151442_comp_novel_subj_3-1_and_3-3_an...,"[3.1, 3.3, 4.2, 4.4]",2932501,66530053,1284645,1484649,[1305244 1325047],[1416451 1423051]
0,20240401_151442_comp_novel_subj_3-1_and_3-3_an...,3278,3477,[3440 3464] [3464 3476],[3466 3522],20240401_151442_comp_novel_subj_3-1_and_3-3_an...,"[3.1, 3.3, 4.2, 4.4]",2932501,66530053,3284672,3484671,[3447271 3470274] [3471071 3483276],[3473674 3529472]
0,20240401_151442_comp_novel_subj_3-1_and_3-3_an...,4476,4675,,,20240401_151442_comp_novel_subj_3-1_and_3-3_an...,"[3.1, 3.3, 4.2, 4.4]",2932501,66530053,4484687,4684691,,
0,20240401_151442_comp_novel_subj_3-1_and_3-3_an...,5474,5673,[5446 5476] [5486 5507] [5530 5552] [5566 5568...,,20240401_151442_comp_novel_subj_3-1_and_3-3_an...,"[3.1, 3.3, 4.2, 4.4]",2932501,66530053,5484697,5684696,[5457296 5487694] [5496701 5517699] [5541504 5...,


In [24]:
exploded_TRODES_METADATA_DF.tail()

,session_dir,tone_start_frame,tone_stop_frame,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
69,20240412_161135_comp_novel_subj_4-2_and_4-4_an...,55593,55794,[55587 55608] [55608 55611] [55633 55637] [556...,[55604 55606] [55606 55616] [55616 55623] [556...,20240412_161135_comp_novel_subj_4-2_and_4-4_an...,"[4.2, 4.4, 5.2, 5.3]",4542881,67770406,55613392,55813392,[55607392 55627992] [55628592 55631390] [55653...,[55623992 55625192] [55626392 55635990] [55636...
69,20240412_161135_comp_novel_subj_4-2_and_4-4_an...,56592,56791,[56579 56619] [56650 56651] [56651 56655] [566...,[56583 56606] [56623 56626] [56626 56637] [566...,20240412_161135_comp_novel_subj_4-2_and_4-4_an...,"[4.2, 4.4, 5.2, 5.3]",4542881,67770406,56613404,56813407,[56601002 56641602] [56671805 56672605] [56673...,[56604804 56627804] [56646003 56648005] [56648...
69,20240412_161135_comp_novel_subj_4-2_and_4-4_an...,57989,58188,[58026 58026] [58051 58052] [58052 58063] [580...,[57996 57997] [57999 58010] [58033 58044] [580...,20240412_161135_comp_novel_subj_4-2_and_4-4_an...,"[4.2, 4.4, 5.2, 5.3]",4542881,67770406,58013422,58213425,[58050222 58051022] [58075023 58076423] [58077...,[58020222 58021622] [58022822 58034422] [58057...
69,20240412_161135_comp_novel_subj_4-2_and_4-4_an...,60185,60385,[60201 60212] [60234 60244] [60246 60248] [602...,[60206 60208] [60208 60217] [60219 60223] [602...,20240412_161135_comp_novel_subj_4-2_and_4-4_an...,"[4.2, 4.4, 5.2, 5.3]",4542881,67770406,60213450,60413452,[60229250 60240650] [60262850 60271848] [60274...,[60234049 60235650] [60236450 60245453] [60247...
69,20240412_161135_comp_novel_subj_4-2_and_4-4_an...,61282,61483,[61227 61306] [61327 61340] [61340 61363] [613...,[61303 61305] [61305 61320] [61321 61322] [613...,20240412_161135_comp_novel_subj_4-2_and_4-4_an...,"[4.2, 4.4, 5.2, 5.3]",4542881,67770406,61313464,61513466,[61257465 61337063] [61357462 61370461] [61371...,[61333064 61335064] [61335461 61350864] [61352...


In [25]:
exploded_TRODES_METADATA_DF.to_csv("./proc/{}_per_video_trial_labels.csv".format(OUTPUT_PREFIX))